# [MCB 32]: Lab 06 - Frog Heart Study

---

### Professor Robin Ball 
Using what you learned, you will create tables for the data you collected in this lab as well as the averaged class data. You will also graph the class data, so it is easier to visualize and find major trends. 

*Estimated Time: ~60 Minutes*

---

### Table of Contents
1. [Creating a Bar Graph](#section_1)

2. [Data Analysis](#section_2)

    a. [Entering your data](#section_2a)
    
    b. [Calculating class data](#section_2b)
    
    c. [Graphing class data](#section_2c)


In [ ]:
%matplotlib inline

from datascience import *
import numpy as np
import matplotlib.pyplot as plt
from qgrid_bridge import *
# from gspread_import import *

---

# 1. Creating a Bar Graph

You will learn how to create bar graphs to help you vizualize the data. We will draw bar graphs with horizontal bars because it’s easier to label the bars that way. Let's start by creating a random table called groceries.

In [ ]:
item_name = ["Apples", "Bananas", "Carrots"]
quantities = [5, 7, 2]
prices = [0.20, 0.25, 0.75]
weights = [0.33, 0.25, 0.16]

In [ ]:
groceries = Table().with_columns(
            "Item Name", item_name, 
            "Quantity", quantities, 
            "Price", prices,
            "Weight", weights)
groceries

In this section, we will draw a graph of the distribution of quantities in the column `Quantity` and weights in `Weight` from the `groceries` table.
 - `select` is a handy method that creates a table with only the desired columns.
 
We don't want the column `Price`, so we only select `Item Name`, `Quantity`, and `Weight`.

In [ ]:
groceries.select("Item Name", "Quantity", "Weight")

To create the bar graph, we will use `barh`.

 - `barh` generates a bar graph of the values in a column. The argument is the column label of the categories. You can create a bar graph using `Table.barh(COLUMN_LABEL)`.
 
Let's plot the new `groceries` table, using `Item Name` as the column label.

In [ ]:
groceries.select("Item Name", "Quantity", "Weight").barh("Item Name")

# 2. Data Analysis

You will use what you learned last week to create tables for this lab. If you need a refresher, you can always refer back to the previous lab notebooks.

### 2a. Entering your data

<div class="alert alert-block alert-info">
    
**QUESTION:** Create tables for Table 1, 2, and 3 that contain the data you collected. For the sake of efficiency, "HR" refers to the heart rate and "amp" refers to the amplitude of the R wave. The labels (column names) of each table have been saved for you in lists, so you can see where to enter and save your table data.

</div>

#### Table 1: Temperature Effects

In [ ]:
table_1 = generate_grid(5, 3, ["Condition", "HR", "amp"], preset_values = {
    "Condition" : ["Human HR", "Basefile frog HR", "Warm temperature", "Baseline before cold",
                  "Cold temperature"]
})
table_1

#### Table 2: Neurotransmitter Effects

In [ ]:
table_2 = generate_grid(4, 3, ["Condition", "HR", "amp"], preset_values = {
    "Condition" : ["Baseline (before epinephrine)", "+ epinephrine", "After wash out (new baseline)", 
                   "+ acetylcholine"]
})
table_2

#### Table 3: Effects of Electrolytes

In [ ]:
table_3 = generate_grid(4, 3, ["Condition", "HR", "amp"], preset_values = {
    "Condition" : ["Baseline (before KCl)", "+ KCl", "Baseline (before CaCl2)", "+ CaCl2"]
})
table_3

Run the cell below to extract the data from your spreadsheets.

In [ ]:
table_1 = get_table(table_1)
table_2 = get_table(table_2)
table_3 = get_table(table_3)

### Accessing an Element in a List

So far, lists have been helpful when it comes to creating tables, but what if you want to access only the first element in the list? Here's how:

In [ ]:
example_list = [1, 2, 3, 4, 5]
example_list[0]

Using square brackets `[ ]` allows you to grab a specific element. In Python, lists start at 0, rather than 1. This is why `example_list[0]` gives you the first element, `1`.

#### Table 4: Change from Baseline
<div class="alert alert-info">
    
**QUESTION:** Using the data from Tables 1, 2, and 3, create Table 4 to show the change in heart rate and R wave amplitude from the baseline.

</div>

In [ ]:
table_4 = generate_grid(6, 3, ["Condition", "HR", "amp"], preset_values = {
    "Condition" : ["Warm temperature", "Cold Temperature", "Epinephrine", "Acetylcholine", "KCl", "CaCl2"]
})
table_4

<div class="alert alert-block alert-warning">

Make sure to input the data from Table 4 into the shared spreadsheet. **Do not continue this lab until everyone has put their data into the shared spreadsheet.**

</div>

### 2b. Calculating class data

In order to make the class data available to you, we will pull it in from Google Sheets using `gspread`. To complete this process, **paste the share link for the spreadsheet into the string below** and run the cell.

In [ ]:
url = ""
sheet_name = ""

class_data = get_spreadsheet(url, sheet_name)
class_data

Now that we have all of the class data, we would like to group the heart rates and amplitudes and get some summary statistics on these groups. To group data, we use the `Table.group()` method which takes two arguments: the name of a column and an _aggregator function_ to which an array of all of the values in a group will be passed.

This means that if we wanted the mean HR of the groups in the `Condition` column, we would make a call like the one below:

In [ ]:
averages = class_data.group("Condition", np.mean)
averages

You should now have a table where each cell represents the mean of the HRs/amplitudes for the specified condition.

<div class="alert alert-info">

**QUESTION:** Create another table, `sdevs`, with the standard deviations for each `Condition` group by filling in the function call below.

</div>

_Hint:_ The function `np.std` returns the standard deviation of an array of values.

In [ ]:
sdevs = class_data.group(..., ...)
sdevs

Finally, we want to concatenate these tables into a single table which will have a single `Condition` column, a column each for the HR mean and standard deviation, and a column each for the amplitude mean and standard deviation. This is accomplished using `Table.join()`:

In [ ]:
class_data = averages.join("Condition", sdevs)

### 2c. Graphing class data

Before we have you graph the class data, let's consider how to add error bars to a bar graph. Consider the bar plot below using our `groceries` table from earlier.

In [ ]:
groceries.drop("Quantity").barh("Item Name")

Let's say we want to (quite arbitrarily) include error bars to $\pm$ 20% of the values shown in the bar graph. We would start by adding these errors to the table:

In [ ]:
groceries = groceries.with_columns(
    "Price Error", [.2 * i for i in groceries.column("Price")],
    "Weight Error", [.2 * i for i in groceries.column("Weight")]
)
groceries

To generate a plot with error bars, we need to use the `matplotlib` code that you've learned about in previous labs. The code below generates a nice barplot, like the one that `Table.barh()` makes, which is why it's so verbose.

In [ ]:
# defining the colors that the datascience package provides for us
blue = (0.0, 30/256, 66/256)
blue = tuple((x + 0.7)/2 for x in blue)
gold = (1.0, 200/256, 44/256)

# generating the bar plot
plt.barh(np.arange(groceries.num_rows), groceries.column("Price"), height=0.4, 
         xerr=groceries.column("Price Error"), color=blue)
plt.barh(np.arange(groceries.num_rows ) - 0.4, groceries.column("Weight"), height=0.4, 
         xerr=groceries.column("Weight Error"), color=gold)
plt.yticks([-0.25, .75, 1.75], labels=groceries.column("Item Name"))
plt.xlabel("Value")
plt.ylabel("Item Name")
plt.legend(["Price", "Weight"], loc=1, bbox_to_anchor=(1.3, 1));

The essential part of the code is adding the `xerr` parameter to the `plt.barh()` call. This is an array of the errors for each value along the $y$-axis.

#### Graph 1: Change in Heart Rate and Standard Deviation Error (from Class Data)

<div class="alert alert-block alert-info">
    
**QUESTION:** Use the table we just created to make a bar graph of the average data for heart rate and amplitude. _Include error bars using the standard deviation as the error._

</div>

_Hint:_ You already have the error information in the `class_data` table since we put the standard deviations in earlier. How can you adapt the code above to use data and column names from that table instead of the `groceries` table?

In [ ]:
...

---
### Saving the Notebook as a PDF

Congrats on finishing your first lab notebook! To turn in this lab assignment follow the steps below:

>1. Press `Control + P` (or `Command + P` on Mac) to open the Print preview
2. Change the destination so that it saves locally on your own computer.
3. Save as PDF
4. If you are stuck, follow further instructions [here](https://www.wikihow.com/Save-a-Web-Page-as-a-PDF-in-Google-Chrome).

Your lab instructor will explain to you what to do afterwards.
---
#### References

- UC Berkeley Foundations of Data Science (Data 8) Textbook: https://www.inferentialthinking.com/chapters/intro

---

Notebook developed by: Samantha Yuan and Chris Pyles

Data Science Modules: http://data.berkeley.edu/education/modules
